# Import dependencies

In [2]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Building an Environment
Build an agent to give us the best shower possible
Randomly temperature
Optimal is 37 to 39 degrees --> Agent doesn't know it 


In [38]:
class ShowerEnv(Env):
    def __init__(self):
        # isn't it two? Like increase or decrease temp?
        self.action_space = Discrete(3)
        #I think this is for the temperature
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Episode length
        self.shower_length = 60

    def step(self, action):
        self.state += action-1

        self.shower_length -= 1

        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1

        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}
        truncated = False

        return self.state, reward, done, info, 

    # This is used a lot in the gaming function of code
    # along with the observation space and the reward space
    def render(self):
        pass

    def reset(self):
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        self.shower_length = 60

        return self.state

# Test Environment

In [39]:
env = ShowerEnv()


In [40]:
env.observation_space

Box(0.0, 100.0, (1,), float32)

In [41]:
env.action_space

Discrete(3)

In [42]:
episodes = 10
for episode in range(episodes+1):
    obs = env.reset()
    terminated = False
    score = 0
    while not terminated:
        env.render()
        action = env.action_space.sample()
        obs, reward, terminated, info = env.step(action)
        score += reward
        if terminated:
            observation = env.reset()
                    
    print(f"Episode:{episode} Score:{score}")

env.close()

Episode:0 Score:-60
Episode:1 Score:-50
Episode:2 Score:-58
Episode:3 Score:-36
Episode:4 Score:36
Episode:5 Score:-52
Episode:6 Score:-48
Episode:7 Score:-60
Episode:8 Score:-52
Episode:9 Score:-50
Episode:10 Score:-26


# Train the model

In [43]:
log_path = os.path.join("training", "Logs")
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log = log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [46]:
model.learn(total_timesteps=40000)

Logging to training/Logs/PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -3.76    |
| time/              |          |
|    fps             | 1412     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -2.91       |
| time/                   |             |
|    fps                  | 1069        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009843727 |
|    clip_fraction        | 0.114       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.916      |
|    explained_variance   | -0.00516    |

In [49]:
save_path = os.path.join("training", "Saved Models", "Shower Model PPO")

In [50]:
model.save(save_path)

/home/sawaiz/anaconda3/envs/rl/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")
